In [1]:
import os
from datetime import datetime
import pandas as pd
import piexif
from fractions import Fraction
from csv import DictWriter
from pixhawk_telelemtry_import import Telemetry


In [24]:

# Get input folder, position file, and output destination from the user
image_folder = r"/Users/matthewtoberman/Downloads/plot_A_test"
# ROV .tlog file
tlog_file_name = r"/Users/matthewtoberman/Downloads/2024-03-02 10-24-00.tlog"
output_csv_file = r"/Users/matthewtoberman/Downloads/WL_global_positions_20240112-140608_TEST.csv"

# ROV
ROV_CLOCK_HOUR_ADJUST=0
# check if .csv file already exists
if tlog_file_name.split('\\')[-1].split('.')[0]+'.csv' not in os.listdir('\\'.join(tlog_file_name.split('\\')[0:-1])):
    combined_CSV_output_filename = None
    Telemetry.logs_to_csv(combined_CSV_output_filename, [tlog_file_name],Telemetry.DEFAULT_FIELDS)
    rov_log_df = Telemetry.csv_to_df(tlog_file_name[0:-4]+'csv')
    rov_log_df.index = rov_log_df.index + pd.DateOffset(hours=ROV_CLOCK_HOUR_ADJUST)

rov_log_df = Telemetry.csv_to_df(tlog_file_name[0:-4]+'csv')


#  remove strange australian time zone
if rov_log_df.index[1].tzinfo != 'none':
    rov_log_df.index = rov_log_df.index.tz_convert(None)


rov_log_df['lat_test'] =  [len(str(decimal)) for decimal in  list(rov_log_df['GLOBAL_POSITION_INT.lat'])]

rov_log_valid_positions_df  = rov_log_df[rov_log_df['lat_test'] > 3]

# create python date time
# rov_log_valid_positions_df ['date_time']=pd.to_datetime(rov_log_valid_positions_df ['SYSTEM_TIME.time_unix_usec'], unit='us', origin='unix')


# interpolate using mean pixhawk data to 1 hz matchting UGPS
mav_df_filtered_1s = rov_log_valid_positions_df.copy()
mav_df_filtered_1s.sort_index(inplace=True, ascending=True)
mav_df_filtered_1s = mav_df_filtered_1s.resample('1S').mean()
mav_df_filtered_1s= mav_df_filtered_1s.interpolate(method ='linear', limit_direction ='backward', limit = 10).copy()
print(f'Pixhawk time span : {mav_df_filtered_1s.index[0]} -  {mav_df_filtered_1s.index[-1]}')


In [31]:


# Open the file for append
position_output_file = open(output_csv_file, 'w', newline='')   

position_headers = ['file name', 'longitude','latitude', 'depth','rov_time','image_time']
writer = DictWriter(position_output_file, delimiter=',', lineterminator='\n',fieldnames=position_headers)
writer.writeheader()
position_output_file.close()



for filename in os.listdir(image_folder):
        if filename.endswith(".JPG") or filename.endswith(".png") or filename.endswith(".jpg"):
           
            image_path = os.path.join(image_folder, filename)

            exif_dict = piexif.load(image_path)

            datetime_object = datetime.strptime(str(exif_dict['Exif'][36867])[2:-1], '%Y:%m:%d %H:%M:%S')
            print(datetime_object)


         
            idx = mav_df_filtered_1s.index.get_indexer([datetime_object], method='nearest')
   
            diff =  pd.to_datetime(mav_df_filtered_1s.index[idx]) - datetime_object
            # print(mav_df_filtered_1s.index[idx])
            # print(datetime_object)
            
            if ((diff.total_seconds()).to_numpy()) < 2:

                position_data_dict = {'file name' : filename , 
                'longitude' :mav_df_filtered_1s['GLOBAL_POSITION_INT.lon'][idx].values[0]/10000000,
                'latitude' :mav_df_filtered_1s['GLOBAL_POSITION_INT.lat'][idx].values[0]/10000000,
                'depth' : mav_df_filtered_1s['VFR_HUD.alt'][idx].values[0],
                'rov_time' : mav_df_filtered_1s.index[idx].values[0],
                'image_time' :datetime_object }
            else:
                print('no match')
                position_data_dict = {'file name' : filename , 
                'longitude' :'',
                'latitude' :'',
                'depth' : '',
                'rov_time' : '',
                'image_time' :'' } 

        
            with open(output_csv_file, mode='a')  as f_filtered:
                dictwriter_object = DictWriter(f_filtered, fieldnames=position_headers)
                dictwriter_object.writerow(position_data_dict)
                # Close the file object
                f_filtered.close ()

         

2024-01-11 11:50:45
